# LSTM for Time Series

The neural network models, based on fully connected layers, that we used in previous examples to forecast the traffic volume do not consider the order of the observations. The observations are implicitely assumed to be indipendent. We know that an observation made at time $t$ depends on past observations. A neural network based on a stack of fully connected layers is not able to memorize and use previous information. For this reason other architectures have been developed to handle ordered sequences of data. 

## Recurrent Neural Network (RNN)
A RNN is a model in which each unit of a layer takes as input an element of the input sequence and the output of the previous unit. The output of each unit is copied and passed to the next unit as shown in the figure. The number of units depends on the length of the sequences but is usually set to a multiple of 2, e.g. 32. The number of RNN layers depends on the complexity of the sequence.

![RNN](images/recurrent_neural_network.svg)

## Long Short-Term Memory Network (LSTM)
A LSTM is a type of RNN that adds a cell state as input to the element of input sequence and the output of the previous unit. A LSTM unit apply several activation functions and operations to those inputs as shown in the figure. These operations allow the model to learn which elements in the sequence are more useful to forecast the next element. All the functional element within a LSTM unit are implemented as neural network layers so that they can be trained using the same algorithms.

![LSTM](images/long_short-term_memory.svg)

In our case, with the traffic data, we will use sequences of 24 hourly observations. Each element $X_t$ of the input sequence is a set of feature values: cloud coverage, day_sin, day_cos, and temperature. The corresponding label is the traffic volume. 

In [1]:
import numpy as np
import pandas as pd
import requests
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import tensorflow as tf
import keras
from keras import layers
print('NumPy version: %s'%np.__version__)
print('Pandas version: %s'%pd.__version__)
print('Matplotlib version: %s'%mpl.__version__)
print('TensorFlow version: %s'%tf.__version__)
print('Keras version: %s'%keras.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
def download_data(url):
  r = requests.get(url)
  data_path = url.rsplit('/')[-1]
  with open(data_path, 'wb') as f:
    f.write(r.content)
  return data_path

In [ ]:
train_url = 'https://raw.githubusercontent.com/luigiselmi/timeseries/main/data/metro_interstate_traffic/train.csv'
val_url = 'https://raw.githubusercontent.com/luigiselmi/timeseries/main/data/metro_interstate_traffic/val.csv'
test_url = 'https://raw.githubusercontent.com/luigiselmi/timeseries/main/data/metro_interstate_traffic/test.csv'
train_data = download_data(train_url)
val_data = download_data(val_url)
test_data = download_data(test_url)
train_df = pd.read_csv(train_data, index_col=0)
val_df = pd.read_csv(val_data, index_col=0)
test_df = pd.read_csv(test_data, index_col=0)

print(train_df.shape, val_df.shape, test_df.shape)

## References
* [Colah - Understanding LSTM networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)